# Project 2: Customer Churn Prediction - Ensembles vs. Deep Learning

**Objective:** Predict customer churn using ensemble methods (Random Forest, XGBoost) and a Deep Neural Network, and compare their performance on a tabular dataset.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Imbalance handling
from imblearn.over_sampling import SMOTE

# Models
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Note: To run this, first download the dataset from Kaggle and place 'WA_Fn-UseC_-Telco-Customer-Churn.csv' in the same directory.
# https://www.kaggle.com/datasets/blastchar/telco-customer-churn

## 2. Load and Explore the Data (EDA)

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

print("First 5 rows:")
print(df.head())
print("\nInfo:")
df.info()

# Drop customerID as it's not a useful feature
df = df.drop('customerID', axis=1)

# Convert TotalCharges to numeric, coercing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Handle missing values (e.g., new customers with 0 tenure might have no TotalCharges)
df.dropna(inplace=True)

# Convert target variable to binary
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

print("\nChurn Distribution:")
print(df['Churn'].value_counts())
sns.countplot(x='Churn', data=df)
plt.title('Churn Distribution')
plt.show()

## 3. Data Preprocessing and Splitting

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

# Identify categorical and numerical features
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

print(f"Numerical features: {numerical_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 4. Preprocess Data and Handle Imbalance with SMOTE

In [ ]:
# Fit the preprocessor on the training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Shape of training data before SMOTE: {X_train_processed.shape}")
print(f"Class distribution before SMOTE: \n{y_train.value_counts()}")

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_processed, y_train)

print(f"\nShape of training data after SMOTE: {X_train_smote.shape}")
print(f"Class distribution after SMOTE: \n{y_train_smote.value_counts()}")

## 5. Model Training

### Model 1: Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_smote, y_train_smote)

### Model 2: XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_smote, y_train_smote)

### Model 3: Deep Neural Network (DNN)

In [ ]:
def build_dnn_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

dnn_model = build_dnn_model(X_train_smote.shape[1])
dnn_model.summary()

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Note: It's good practice to create a validation set from the SMOTE'd training set for the DNN
X_train_dnn, X_val_dnn, y_train_dnn, y_val_dnn = train_test_split(X_train_smote, y_train_smote, test_size=0.1, random_state=42)

history = dnn_model.fit(
    X_train_dnn,
    y_train_dnn,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_dnn, y_val_dnn),
    callbacks=[early_stopping],
    verbose=0
)

## 6. Model Evaluation

In [ ]:
models_to_evaluate = {
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'DNN': dnn_model
}

results = {}

for name, model in models_to_evaluate.items():
    print(f"--- Evaluating {name} ---")
    if name == 'DNN':
        y_pred_proba = model.predict(X_test_processed).ravel()
        y_pred = (y_pred_proba > 0.5).astype(int)
    else:
        y_pred = model.predict(X_test_processed)
        y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC AUC Score: {roc_auc:.4f}\n")
    
    results[name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba, 'roc_auc': roc_auc}
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

## 7. Comparison and Conclusion

In [ ]:
plt.figure(figsize=(10, 8))

for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
    plt.plot(fpr, tpr, label=f"{name} (AUC = {result['roc_auc']:.2f})")

plt.plot([0, 1], [0, 1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid()
plt.show()

print("Conclusion: For this structured dataset, XGBoost provides a slight edge in performance over Random Forest and the DNN. \n",
      "This highlights that for many tabular data problems, gradient boosting models are an extremely strong, often state-of-the-art, choice.")